In [1]:
# python 2.7
# certifi(2016.9.26) intervaltree(3.1.0) MSIpred(0.1) numpy(1.16.6) pandas(0.20.3) pip(20.0.2) python-dateutil(2.8.1) pytz(2020.4) 
# scikit-learn(0.19.2) scipy(1.2.3) setuptools(36.4.0) six(1.15.0) sklearn(0.0) sortedcontainers(2.3.0) wheel(0.35.1)
import os,shutil
import gzip

In [2]:
# Get .maf.gz file path
wenjian= "../data/maf/COAD_maf"
maf_name=[]
root_maf_name=[]
for root,dirs,files in os.walk(wenjian):
    for f in files:
        if 'maf.gz' in f and '.parcel' not in f:
            maf_name.append(f)
            f=os.path.join(root,f)
            root_maf_name.append(f)

In [6]:
# maf_name
# len(maf_name) # 455
# root_maf_name
# len(root_maf_name) # 455

['../data/maf/COAD_maf/852e7b78-a964-4b05-b2c9-d805afc3b7d9/8307059c-97b6-47fd-a4ea-95398235a08c.wxs.aliquot_ensemble_masked.maf.gz',
 '../data/maf/COAD_maf/80ad18e8-cca2-4337-8557-5d16fca26e30/55a29957-c76c-4db6-94da-f76169ba74d5.wxs.aliquot_ensemble_masked.maf.gz',
 '../data/maf/COAD_maf/cd15e504-2c07-4df6-894a-7f4cc336c1c8/8ce4787c-f780-40bd-8f3b-2872f34bb440.wxs.aliquot_ensemble_masked.maf.gz',
 '../data/maf/COAD_maf/bb72d94d-6bdc-4459-a1f6-9da91315a01e/122ab28a-a0c3-4fee-a157-0d0ac7ba6eb3.wxs.aliquot_ensemble_masked.maf.gz',
 '../data/maf/COAD_maf/8056e60e-c56c-4bec-9af8-494c46ce4b6e/23a2a575-765c-4b41-ad29-c905031fdef1.wxs.aliquot_ensemble_masked.maf.gz',
 '../data/maf/COAD_maf/8e55f9a1-7574-40bc-bc6b-5221688016f4/e7ab9dfb-910e-4979-a920-6c1568fd9d44.wxs.aliquot_ensemble_masked.maf.gz',
 '../data/maf/COAD_maf/0c0928ab-707f-4615-9a27-e0a72e128c64/1491b2a0-6f48-4b18-b46b-be6b65de3d4f.wxs.aliquot_ensemble_masked.maf.gz',
 '../data/maf/COAD_maf/476ed192-fd7c-4274-a719-c733d9f81c28/97

In [4]:
# Define a decompression function
def un_gz(file_name):  
    # Get the name of the file, remove the suffix
    f_name = file_name.replace(".gz", "")
    # start decompression
    g_file = gzip.GzipFile(file_name)
    # Read the decompressed file and write the same name file with the suffix removed (that is, get the decompressed file)
    open(f_name, "wb+").write(g_file.read())
    g_file.close()

In [5]:
for file in root_maf_name:
    un_gz(file)

In [8]:
# import maf file
import pandas as pd
import MSIpred as mp
import scipy as sp

In [8]:
# get maf file path
wenjian= "../data/maf/COAD_maf"
maf_name=[]
root_maf_name=[]

for root,dirs,files in os.walk(wenjian):
    for f in files:
        if '.maf' in f and '.gz' not in f:
            maf_name.append(f)
            f=os.path.join(root,f)
            root_maf_name.append(f)

In [9]:
# maf_name
# len(maf_name) # 455
# root_maf_name
# len(root_maf_name) # 455

455

In [ ]:
# Predict each maf file through the MSIpred package and save it to a csv file

In [49]:
for f in root_maf_name:
    data = mp.Raw_Maf(maf_path=f)
    # Generate an annotated maf file (tagged_maf_file) by adding an In_repeats column
    # For further analysis (specify simpleRepeats.txt, whether the mutation occurs in the simple repeat region), this step will be slow and requires patience
    data.create_tagged_maf(ref_repeats_file='../data/maf/simpleRepeat.txt',tagged_maf_file = 'tagged_data.maf')
    # Using the annotated MAF file (tagged_crc.maf) generated in the previous step, initialize
    tagged_data_maf = mp.Tagged_Maf(tagged_maf_path='tagged_data.maf')
    # Create feature dadaframe from generated maf file for further MSI prediction to obtain Mb of exon sequence (COAD project, exon size is 44Mb)
    data_features = tagged_data_maf.make_feature_table(exome_size=44)
    # Using the SVM classifier to predict MSI status (MSS or MSI-H) in the feature dataframe, you will get a pandas dataframe including the predicted MSI status
    predicted_MSI = mp.msi_prediction(feature_table=data_features)
    # Save the generated predicted_MSI dataframe
    pd.DataFrame.to_csv(predicted_MSI,'../data/maf/COAD_data_MSI.csv',sep=',',header=False,mode='a')

In [ ]:
# Read the generated forecast msi data

In [9]:
data_MSI = pd.read_csv(r"../data/maf/COAD_data_MSI.csv",sep = ',',encoding = 'utf-8',usecols=['Predicted_MSI_Status','simple_tumor'])

In [73]:
# Convert the above dataframe into a list format so that you can use regular expressions to take out the sample names
data_MSI_list = data_MSI.values.tolist()
data_MSI_list
len(data_MSI_list) # 454

[[u'TCGA-A6-6781-01A-22D-1924-10', u'MSI-H', u'TCGA-A6-6781-01A'],
 [u'TCGA-CM-6172-01A-11D-1650-10', u'MSS', u'TCGA-CM-6172-01A'],
 [u'TCGA-DM-A0XD-01A-12D-A152-10', u'MSS', u'TCGA-DM-A0XD-01A'],
 [u'TCGA-AA-3655-01A-02D-1719-10', u'MSS', u'TCGA-AA-3655-01A'],
 [u'TCGA-DM-A1HA-01A-11D-A152-10', u'MSS', u'TCGA-DM-A1HA-01A'],
 [u'TCGA-D5-7000-01A-11D-1924-10', u'MSS', u'TCGA-D5-7000-01A'],
 [u'TCGA-F4-6856-01A-11D-1924-10', u'MSI-H', u'TCGA-F4-6856-01A'],
 [u'TCGA-AA-3667-01A-01W-0900-09', u'MSS', u'TCGA-AA-3667-01A'],
 [u'TCGA-D5-6538-01A-11D-1719-10', u'MSS', u'TCGA-D5-6538-01A'],
 [u'TCGA-AZ-6600-01A-11D-1771-10', u'MSS', u'TCGA-AZ-6600-01A'],
 [u'TCGA-G4-6317-01A-11D-1719-10', u'MSS', u'TCGA-G4-6317-01A'],
 [u'TCGA-AZ-4315-01A-01D-1408-10', u'MSS', u'TCGA-AZ-4315-01A'],
 [u'TCGA-AA-3955-01A-02W-0995-10', u'MSS', u'TCGA-AA-3955-01A'],
 [u'TCGA-AA-3520-01A-01D-1953-10', u'MSS', u'TCGA-AA-3520-01A'],
 [u'TCGA-CM-6168-01A-11D-1650-10', u'MSS', u'TCGA-CM-6168-01A'],
 [u'TCGA-AA-3811-01A-

In [74]:
import re
# Use regular expression to take the first few of the tumor
# Normally, just take the first four of the previous sample name, you can check the sample name definition for details.
pattern = re.compile(r"(TCGA-\w*-\w*-\w*)")  
MSI_list = []
for strs in data_MSI_list:
    test_strs = "".join(strs)
    result = pattern.findall(test_strs)
    MSI_list.append("".join(result))
MSI_list

[u'TCGA-A6-6781-01ATCGA-A6-6781-01A',
 u'TCGA-CM-6172-01ATCGA-CM-6172-01A',
 u'TCGA-DM-A0XD-01ATCGA-DM-A0XD-01A',
 u'TCGA-AA-3655-01ATCGA-AA-3655-01A',
 u'TCGA-DM-A1HA-01ATCGA-DM-A1HA-01A',
 u'TCGA-D5-7000-01ATCGA-D5-7000-01A',
 u'TCGA-F4-6856-01ATCGA-F4-6856-01A',
 u'TCGA-AA-3667-01ATCGA-AA-3667-01A',
 u'TCGA-D5-6538-01ATCGA-D5-6538-01A',
 u'TCGA-AZ-6600-01ATCGA-AZ-6600-01A',
 u'TCGA-G4-6317-01ATCGA-G4-6317-01A',
 u'TCGA-AZ-4315-01ATCGA-AZ-4315-01A',
 u'TCGA-AA-3955-01ATCGA-AA-3955-01A',
 u'TCGA-AA-3520-01ATCGA-AA-3520-01A',
 u'TCGA-CM-6168-01ATCGA-CM-6168-01A',
 u'TCGA-AA-3811-01ATCGA-AA-3811-01A',
 u'TCGA-AY-6196-01ATCGA-AY-6196-01A',
 u'TCGA-CM-6680-01ATCGA-CM-6680-01A',
 u'TCGA-DM-A0XF-01ATCGA-DM-A0XF-01A',
 u'TCGA-CM-6170-01ATCGA-CM-6170-01A',
 u'TCGA-AA-3939-01ATCGA-AA-3939-01A',
 u'TCGA-F4-6569-01ATCGA-F4-6569-01A',
 u'TCGA-A6-2672-01BTCGA-A6-2672-01B',
 u'TCGA-AY-A69D-01ATCGA-AY-A69D-01A',
 u'TCGA-AA-3842-01ATCGA-AA-3842-01A',
 u'TCGA-A6-A565-01ATCGA-A6-A565-01A',
 u'TCGA-AA-3

In [69]:
data_MSI['simple_tumor4'] = MSI_list

In [70]:
data_MSI

,simple_tumor,Predicted_MSI_Status,simple_tumor4
0,TCGA-A6-6781-01A-22D-1924-10,MSI-H,TCGA-A6-6781-01A
1,TCGA-CM-6172-01A-11D-1650-10,MSS,TCGA-CM-6172-01A
2,TCGA-DM-A0XD-01A-12D-A152-10,MSS,TCGA-DM-A0XD-01A
3,TCGA-AA-3655-01A-02D-1719-10,MSS,TCGA-AA-3655-01A
4,TCGA-DM-A1HA-01A-11D-A152-10,MSS,TCGA-DM-A1HA-01A
5,TCGA-D5-7000-01A-11D-1924-10,MSS,TCGA-D5-7000-01A
6,TCGA-F4-6856-01A-11D-1924-10,MSI-H,TCGA-F4-6856-01A
7,TCGA-AA-3667-01A-01W-0900-09,MSS,TCGA-AA-3667-01A
8,TCGA-D5-6538-01A-11D-1719-10,MSS,TCGA-D5-6538-01A
9,TCGA-AZ-6600-01A-11D-1771-10,MSS,TCGA-AZ-6600-01A


In [71]:
data_MSI.to_csv("../data/maf/COAD_data_MSI.csv", index=False, sep=',') 